## Section 3 Note: ETL & NLP

### ETL Pipelines

#### Extract: CSV, JSON, XML, SQL DB, API

In [ ]:
# helper, open file to check first n lines
def print_lines(n, file_name):
    f = open(file_name)
    for i in range(n):
        print(f.readline())
    f.close

In [2]:
# CSV
import pandas as pd
df = pd.read_csv('file', dtype=str, skiprows=4)

In [ ]:
# JSON - method 1
import pandas as pd
df = pd.read_json('file', orient='records')
# use orient option based on the file layouts

# JSON - method 2
import json
with open('file') as f:
    js_data = json.load(f)
    
# first record
print(js_data[0])

In [4]:
# XML
from bs4 import BeautifulSoup
with open('file') as fp:
    soup = BeautifulSoup(fp, "lxml") # lxml is the parser type

# check first 5 records
i = 0
for recrod in soup.final_all('record'):
    i += 1
    for record in record.find_all('field'):
        print(record['name'], ': ', record.text)
    print()
    if i == 5:
        break

In [5]:
# SQL - SQLite3
import sqlite3
import pandas as pd

conn = sqlite3.connect('file.db')
pd.read_sql('SELECT * FROM table', conn)

# SQL - SQLAlchemy
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('sqlite:///../path/file.db')
pd.read_sql('SELECT * FROM table', engine)

In [8]:
# API
import requests
import pandas as pd

url = 'http://weblink'
r = requests.get(url)
r.json() # assume retrived file is json

#### Load

In [ ]:
df.to_csv('file.csv', index=False)
df.to_json('file.json', orient='records')

import sqlite3
conn = sqlite3.connect('file.db')
df.to_sql('tablename', con = conn, if_exists='replace', index=False)

# update database
conn = sqlite3.connect('file.db')
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS tablename')
cur.execute("CREATE TABLE tablename (col1 TEXT PRIMARY KEY, col2 REAL, col3 INTEGER);")
cur.execute("INSERT INTO tablename (col1, col2, col3) VALUES ('a', 1.2, 1);")
conn.commit()
conn.close()

# select all
cur.execute("SELECT * FROM tablename")
cur.fetchall()

# helper: assign values from df to colnames
col1, col2, col3 = df.iterrows()

#### Transform: encoding

In [9]:
from encodings.aliases import aliases

# For file unclear what the encoding is
for encoding in set(aliases.values()):
    try:
        df = pd.read_csv('file', encoding=encoding)
        print('successful', encoding)
    except:
        pass

# Or use chardet library to identify
import chardet

with open('file', 'rb') as file:
    print(chardet.detect(file.read()))

In [ ]:
# impute by group
df['col'] = df.sort_values('x').groupby('group')['col'].fillna(method='ffill')

In [ ]:
# Unique value
uni, count = np.unique(df, return_counts=True)
df['col'],unique()

In [ ]:
# Create new cols systematically
def create_newcol(b, k):
# create new_col as 2b, 3b, .. until kb
    new_col = []
    for i in range(2, k+1):
        new_col.append(b**i)
    return new_col

def create_newcol_name(colname, k):
    col_names = []
    for i in range(2, k+1):
        col_names.append('{}{}'.format(colname, i))
    return col_names

def concate_newcol(df, column, k):
    new_feature = df[column].apply(lambda x: create_newcol(x, k))
    new_feature_df = pd.DataFrame(new_feature.tolist(), columns = create_newcol_name(column, k))
    return pd.concat([df, new_feature_df], axis=1)    

### NLP
#### 1. Cleaning
#### 2. Normalizing
#### 3. Tokenizing
#### 4. Stop words
#### 5. Parts of Speech POS tagging
#### 6. Stemming and lemmatization
#### 7. Bag of Words BOW and TF-IDF

In [28]:
# Cleaning
import requests
from bs4 import BeautifulSoup

# get text from a webpage
r = requests.get("https://www.udacity.com/courses/all")

# remove HTML tags
soup = BeautifulSoup(r.text)

# find all summary - based on inspect the code of html
summaries = soup.find_all("div", {"class": "course-summary-card"})
print(len(summaries)) # number of records
print(summaries[0].prettify()) # checck first record

# extract course title and school
course = []
for summ in summaries:
    title = summ.select_one("h3").get_text().strip()
    school = summ.select_one("h4").get_text().strip()
    course.append((title, school))

In [11]:
text = "The first time you see The Second Renaissance it may look boring. Look at it at least twice and definitely watch part 2. It will change your view of the matrix. Are the human people the ones who started the war ? Is AI a bad thing ?"

In [12]:
# Normalization
# lower case
text = text.lower()

# remove punctuation
import re

text = re.sub(r"[^a-zA-Z0-9]", " ", text)
text

'the first time you see the second renaissance it may look boring  look at it at least twice and definitely watch part 2  it will change your view of the matrix  are the human people the ones who started the war   is ai a bad thing  '

In [13]:
# Tokenization
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize, sent_tokenize

words = word_tokenize(text)
sentences = sent_tokenize(text)
print(words)

['the', 'first', 'time', 'you', 'see', 'the', 'second', 'renaissance', 'it', 'may', 'look', 'boring', 'look', 'at', 'it', 'at', 'least', 'twice', 'and', 'definitely', 'watch', 'part', '2', 'it', 'will', 'change', 'your', 'view', 'of', 'the', 'matrix', 'are', 'the', 'human', 'people', 'the', 'ones', 'who', 'started', 'the', 'war', 'is', 'ai', 'a', 'bad', 'thing']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\qz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
# Stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

words = [w for w in words if w not in stopwords.words("english")]
print(words)

['first', 'time', 'see', 'second', 'renaissance', 'may', 'look', 'boring', 'look', 'least', 'twice', 'definitely', 'watch', 'part', '2', 'change', 'view', 'matrix', 'human', 'people', 'ones', 'started', 'war', 'ai', 'bad', 'thing']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\qz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
# POS tag
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize

pt = pos_tag(words)
#print(pt)

# Named Entity Recognition (NER)
tree = ne_chunk(pt)
#print(tree)

# Define a custom grammar
my_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")
parser = nltk.ChartParser(my_grammar)
for tree in parser.parse(word_tokenize("I shot an elephant in my pajamas")):
    print(tree)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


In [25]:
# Stemming and Lemmatizing
import nltk
nltk.download('wordnet') #lemmatization
from nltk.corpus import stopwords

# stemming
from nltk.stem.porter import PorterStemmer
stemmed = [PorterStemmer().stem(w) for w in words]
# print(stemmed)

# lemmatization
from nltk.stem.wordnet import WordNetLemmatizer
lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
#print(lemmed))

# lemmatize verbs by specifying pos
lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]
lemmed

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\qz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['first',
 'time',
 'see',
 'second',
 'renaissance',
 'may',
 'look',
 'bore',
 'look',
 'least',
 'twice',
 'definitely',
 'watch',
 'part',
 '2',
 'change',
 'view',
 'matrix',
 'human',
 'people',
 'one',
 'start',
 'war',
 'ai',
 'bad',
 'thing']

In [48]:
# Bag of Words - CountVectorizer
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

vect = CountVectorizer(tokenizer=tokenize)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\qz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\qz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [51]:
corpus = ["The first time you see The Second Renaissance it may look boring.",
        "Look at it at least twice and definitely watch part 2.",
        "It will change your view of the matrix.",
        "Are the human people the ones who started the war?",
        "Is AI a bad thing ?"]

x = vect.fit_transform(corpus).toarray()
x
vect.vocabulary_

{'first': 6,
 'time': 20,
 'see': 17,
 'second': 16,
 'renaissance': 15,
 'may': 11,
 'look': 9,
 'boring': 3,
 'least': 8,
 'twice': 21,
 'definitely': 5,
 'watch': 24,
 'part': 13,
 '2': 0,
 'change': 4,
 'view': 22,
 'matrix': 10,
 'human': 7,
 'people': 14,
 'one': 12,
 'started': 18,
 'war': 23,
 'ai': 1,
 'bad': 2,
 'thing': 19}

In [53]:
# TF-IDF Transformer
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer(smooth_idf=False)
tfidf = transformer.fit_transform(x).toarray()
tfidf

array([[0.        , 0.        , 0.        , 0.36419547, 0.        ,
        0.        , 0.36419547, 0.        , 0.        , 0.26745392,
        0.        , 0.36419547, 0.        , 0.        , 0.        ,
        0.36419547, 0.36419547, 0.36419547, 0.        , 0.        ,
        0.36419547, 0.        , 0.        , 0.        , 0.        ],
       [0.39105193, 0.        , 0.        , 0.        , 0.        ,
        0.39105193, 0.        , 0.        , 0.39105193, 0.28717648,
        0.        , 0.        , 0.        , 0.39105193, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.39105193, 0.        , 0.        , 0.39105193],
       [0.        , 0.        , 0.        , 0.        , 0.57735027,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.57735027, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.57735027, 0.

In [54]:
# TF-IDF Vectorizer = TF-IDF Tranformer + Count Vectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus).toarray()
X

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.30298183, 0.        , 0.        , 0.30298183, 0.        ,
        0.        , 0.20291046, 0.        , 0.24444384, 0.        ,
        0.30298183, 0.        , 0.        , 0.        , 0.        ,
        0.30298183, 0.30298183, 0.30298183, 0.        , 0.40582093,
        0.        , 0.30298183, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.30298183, 0.        ],
       [0.        , 0.30015782, 0.        , 0.60031564, 0.        ,
        0.        , 0.        , 0.30015782, 0.        , 0.        ,
        0.        , 0.20101919, 0.30015782, 0.24216544, 0.        ,
        0.        , 0.        , 0.        , 0.30015782, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.30015782, 0.        , 0.        ,
        0.30015782, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.